In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. Setup Browser
options = Options()
# options.add_argument("--headless") # Aktifkan ini kalau gak mau liat browser kebuka
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20) # Nunggu maksimal 20 detik

all_jobs = []

try:
    # Cari lowongan Python di Indonesia
    print("Mencari lowongan di JobStreet...")
    driver.get("https://www.jobstreet.co.id/id/Python-jobs")

    # 2. Tunggu sampai card lowongan muncul
    # Kita cari elemen yang punya atribut 'data-automation' (khas JobStreet)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'article')))

    # 3. Ambil semua artikel lowongan
    job_cards = driver.find_elements(By.TAG_NAME, 'article')
    print(f"Ditemukan {len(job_cards)} lowongan di halaman ini.")

    for card in job_cards:
        try:
            # Menggunakan CSS Selector untuk presisi
            title = card.find_element(By.CSS_SELECTOR, 'a[data-automation="jobTitle"]').text
            company = card.find_element(By.CSS_SELECTOR, 'a[data-automation="jobCompany"]').text
            location = card.find_element(By.CSS_SELECTOR, 'a[data-automation="jobLocation"]').text
            
            all_jobs.append({
                "Title": title,
                "Company": company,
                "Location": location
            })
            print(f"Scraped: {title} di {company}")
        except:
            continue

    # 4. Export ke Excel
    df = pd.DataFrame(all_jobs)
    df.to_excel("lowongan_jobstreet.xlsx", index=False)
    print("\nFile 'lowongan_jobstreet.xlsx' berhasil dibuat!")

finally:
    driver.quit()